In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import collections
from collections import defaultdict, Counter
from itertools import chain

In [ ]:
def read_file(file_name):
    with open(f'inputs\\{file_name}', 'r') as f:
        data = f.read().strip()
    return data.split('\n')

In [ ]:
files = ['a.txt', 'b.txt', 'c.txt', 'd.txt', 'e.txt', 'f.txt']

In [ ]:
file_idx = 3
input_data = read_file(files[file_idx])

In [ ]:
input_data

In [ ]:
def parse_input(input_data):
    Bs, Es, streets, Ls, Ps, routes = list(), list(), list(), list(), list(), list()
    for i, line in enumerate(input_data):
        if i == 0:
            D, I, S, V, F = map(int, line.split(' ')) # duration, intersections, streets, cars, bonus points
            print(D, I, S, V, F)
        elif 0 < i < S+1:
            B, E, street, L = line.split(' ')
            Bs.append(int(B))
            Es.append(int(E))
            streets.append(street)
            Ls.append(int(L))
            print(B,E, street, L)
        else:
            split_line = line.split(' ')
            P = split_line[0]
            Ps.append(int(P))
            route = split_line[1:]
            routes.append(route)
            print(P, route)
    return D, I, S, V, F, Bs, Es, streets, Ls, Ps, routes

In [ ]:
D, I, S, V, F, Bs, Es, streets, Ls, Ps, routes = parse_input(input_data)

In [ ]:
count_begins = Counter(Bs)
begins_sorted = dict(sorted(count_begins.items(), key=lambda x: x[1], reverse=True))
begins_sorted

In [ ]:
count_begins

In [ ]:
sum(begins_sorted.values())

In [ ]:
count_ends = Counter(Es)
ends_sorted = dict(sorted(count_ends.items(), key=lambda x: x[1], reverse=True))
ends_sorted

In [ ]:
pd.Series(ends_sorted.values()).describe()

In [ ]:
plt.Figure()
pd.Series(ends_sorted.values()).hist(bins=100)
plt.xlabel('Streets to/from intersection')

In [ ]:
routes

In [ ]:
chained = chain.from_iterable(routes)

In [ ]:
chained = chain.from_iterable(routes)
count_streets = Counter(chained)
street_counts_sorted = dict(sorted(count_streets.items(), key=lambda x: x[1], reverse=True))
street_counts_sorted

In [ ]:
pd.Series(street_counts_sorted.values()).describe()

In [ ]:
plt.figure()
plt.hist(pd.Series(street_counts_sorted.values()), bins=12)
plt.ylabel('#')
plt.xlabel('Number of cars through street')

In [ ]:
sum(Ps)

In [ ]:
sum(street_counts_sorted.values())

In [ ]:
streets

In [ ]:
S = dict(zip(streets, Ls))

In [ ]:
S

In [ ]:
route_length = list()
for route in routes:
    length = 0
    for i, street in enumerate(route):
        if i == 0:
            continue
        length += S[street]
    route_length.append(length)

In [ ]:
plt.figure()
plt.hist(pd.Series(route_length), bins=30)
plt.ylabel('#')
plt.xlabel('Route length (excl. first')

In [ ]:
pd.Series(route_length).describe()

In [ ]:
D, F

In [ ]:
goal = 1.5e6
sum_ = 0

for i, route_len in enumerate(sorted(route_length, reverse=True)):
    sum_ += (D - route_len) + F
    if sum_ > goal:
        print(i)
        break

# drukste kruispunt per route

In [ ]:
street_to_I = dict(zip(streets, Es))

In [ ]:
ends_sorted

In [ ]:
street_to_I

In [ ]:
busiest_intersection_on_routes = list()
for route in routes:
    busiest = 0
    for i, street in enumerate(route):
        n_streets_end_at_intersection = ends_sorted[street_to_I[street]]
        if n_streets_end_at_intersection > busiest:
            busiest = n_streets_end_at_intersection
    busiest_intersection_on_routes.append(busiest)

In [ ]:
pd.Series(busiest_intersection_on_routes).describe()

In [ ]:
plt.figure()
plt.hist(pd.Series(busiest_intersection_on_routes), bins=30)
plt.xlabel('Busiest intersection on route')

# aankomsttijd per straat op drukste kruispunt

In [ ]:
intersection_to_review = 8
street_to_I = dict(zip(streets, Es))
street_lengths = dict(zip(streets, Ls))
arrivals_time_and_street = list()
street_to_int_mapping = dict()
for route in routes:
    time = 0
    for i, street in enumerate(route):
        if i != 0:
            time += street_lengths[street]
        
        intersection = street_to_I[street]
        if intersection != intersection_to_review:
            continue
            
        if street in street_to_int_mapping:
            pass
        else:
            street_to_int_mapping[street] = len(street_to_int_mapping.keys())
        
        arrivals_time_and_street.append((time, street_to_int_mapping[street]))

In [ ]:
# some streets to the intersection are not on a route -- len(street_to_int_mapping.keys())

plt.figure()
times, street_ids = zip(*arrivals_time_and_street)
plt.scatter(times, street_ids, s=2)
plt.xlabel(f'arrival time at intersection {intersection_to_review}')
plt.ylabel('street id')
non_zero = len(street_to_int_mapping.keys())
plt.suptitle(f'{non_zero} streets with cars')

In [ ]:
Counter(Counter(street_ids).values())

In [ ]:
arrival_times_via_street = defaultdict(list)

for time, street_id in arrivals_time_and_street:
    arrival_times_via_street[street_id].append(time)

In [ ]:
single_car_street_arrivals = list()
for arrivals in arrival_times_via_street.values():
    if len(arrivals) == 1:
        single_car_street_arrivals += arrivals
    
print(sorted(single_car_street_arrivals))

In [ ]:
# Check if the arrival times are close to each other
differences = list()
multi_car_street_arrivals = list()
for arrivals in arrival_times_via_street.values():
    if len(arrivals) != 2: # == 1
        continue
    
    diff = max(arrivals) - min(arrivals)
    differences.append(diff)
    multi_car_street_arrivals.append((diff, sorted(arrivals)))
pd.Series(differences).describe()
print(sorted(differences))
print(sorted(multi_car_street_arrivals))